# Part-of-Speech Tagging using Feedforward Networks and Embeddings
Author: Pierre Nugues

A part-of-speech tagger using feed-forward networks and GloVe embeddings and trained on a corpus following the Universal Dependencies format. Here we use the English Web Treebank:
https://github.com/UniversalDependencies/UD_English-EWT/tree/master.

## Python Headers

### The Modules

In [ ]:
import sys
import os
from sklearn.feature_extraction import DictVectorizer
import time
from keras import models, layers
import sys
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.models import load_model
import math
from keras.preprocessing.sequence import pad_sequences

### Some Parameters

In [ ]:
OPTIMIZER = 'rmsprop'
BATCH_SIZE = 128
EPOCHS = 4
MINI_CORPUS = False
EMBEDDING_DIM = 100
W_SIZE = 2

## Preprocessing

### Loading the Embeddings
We will use GloVe embeddings and load them

In [ ]:
def load(file):
    """
    Return the embeddings in the from of a dictionary
    :param file:
    :return:
    """
    file = file
    embeddings = {}
    glove = open(file)
    for line in glove:
        values = line.strip().split()
        word = values[0]
        vector = np.array(values[1:], dtype='float32')
        embeddings[word] = vector
    glove.close()
    embeddings_dict = embeddings
    embedded_words = sorted(list(embeddings_dict.keys()))
    return embeddings_dict

In [ ]:
embedding_file = '/Users/pierre/Documents/Cours/EDAN20/corpus/glove.6B.100d.txt'
embeddings_dict = load(embedding_file)

In [ ]:
embeddings_dict['table']

### Loading the Corpus

In [ ]:
def load_ud_en_ewt():

    train_file = '../datasets/ud_en/en_ewt-ud-train.conllu'
    dev_file = '../datasets/ud_en/en_ewt-ud-dev.conllu'
    test_file = '../datasets/ud_en/en_ewt-ud-test.conllu'
    column_names = ['ID', 'FORM', 'LEMMA', 'UPOS', 'XPOS', 
                    'FEATS', 'HEAD', 'DEPREL', 'HEAD', 'DEPS', 'MISC']
    column_names = list(map(str.lower, column_names))
    train_sentences = open(train_file).read().strip()
    dev_sentences = open(dev_file).read().strip()
    test_sentences = open(test_file).read().strip()
    # test2_sentences = open(test2_file).read().strip()
    return train_sentences, dev_sentences, test_sentences, column_names

def load_conll2009_pos():
    train_file = '/Users/pierre/Documents/Cours/EDAN20/corpus/conll2009/en/CoNLL2009-ST-English-train-pos.txt'
    dev_file = '/Users/pierre/Documents/Cours/EDAN20/corpus/conll2009/en/CoNLL2009-ST-English-development-pos.txt'
    test_file = '/Users/pierre/Documents/Cours/EDAN20/corpus/conll2009/en/CoNLL2009-ST-test-words-pos.txt'
    # test2_file = 'simple_pos_test.txt'

    column_names = ['id', 'form', 'lemma', 'plemma', 'pos', 'ppos']

    train_sentences = open(train_file).read().strip()
    dev_sentences = open(dev_file).read().strip()
    test_sentences = open(test_file).read().strip()
    # test2_sentences = open(test2_file).read().strip()
    return train_sentences, dev_sentences, test_sentences, column_names

# train_sentences, dev_sentences, test_sentences, column_names = \
# load_conll2009_pos()
train_sentences, dev_sentences, test_sentences, column_names =\
load_ud_en_ewt()
train_sentences[:100]

### Converting the Corpus in a Dictionary
We follow the fit-transform pattern of sklearn

In [ ]:
import regex as re

class Token(dict):
    pass

class CoNLLDictorizer:

    def __init__(self, column_names, sent_sep='\n\n', col_sep=' +'):
        self.column_names = column_names
        self.sent_sep = sent_sep
        self.col_sep = col_sep

    def fit(self):
        pass

    def transform(self, corpus):
        corpus = corpus.strip()
        sentences = re.split(self.sent_sep, corpus)
        return list(map(self._split_in_words, sentences))

    def fit_transform(self, corpus):
        return self.transform(corpus)

    def _split_in_words(self, sentence):
        rows = re.split('\n', sentence)
        rows = [row for row in rows if row[0] != '#']
        return [Token(dict(zip(self.column_names,
                               re.split(self.col_sep, row))))
                for row in rows]

In [ ]:
conll_dict = CoNLLDictorizer(column_names, col_sep='\t')
train_dict = conll_dict.transform(train_sentences)
if MINI_CORPUS:
    train_dict = train_dict[:len(train_dict) // 15]
test_dict = conll_dict.transform(test_sentences)
print('First sentence, train:', train_dict[0])

### Extracting the Context and Dictorizing it

In [ ]:
class ContextDictorizer():
    """
    Extract contexts of words in a sequence
    Contexts are of w_size to the left and to the right
    Builds an X matrix in the form of a dictionary
    and possibly extracts the output, y, if not in the test step
    If the test_step is True, returns y = []
    """

    def __init__(self, input='form', output='upos', w_size=2, tolower=True):
        self.BOS_symbol = '__BOS__'
        self.EOS_symbol = '__EOS__'
        self.input = input
        self.output = output
        self.w_size = w_size
        self.tolower = tolower
        # This was not correct as the names were not sorted
        # self.feature_names = [input + '_' + str(i)
        #                     for i in range(-w_size, w_size + 1)]
        # To be sure the names are ordered
        zeros = math.ceil(math.log10(2 * w_size + 1))
        self.feature_names = [input + '_' + str(i).zfill(zeros) for 
                              i in range(2 * w_size + 1)]

    def fit(self, sentences):
        """
        Build the padding rows
        :param sentences:
        :return:
        """
        self.column_names = sentences[0][0].keys()
        start = [self.BOS_symbol] * len(self.column_names)
        end = [self.EOS_symbol] * len(self.column_names)
        start_token = Token(dict(zip(self.column_names, start)))
        end_token = Token(dict(zip(self.column_names, end)))
        self.start_rows = [start_token] * self.w_size
        self.end_rows = [end_token] * self.w_size

    def transform(self, sentences, training_step=True):
        X_corpus = []
        y_corpus = []
        for sentence in sentences:
            X, y = self._transform_sentence(sentence, training_step)
            X_corpus += X
            if training_step:
                y_corpus += y
        return X_corpus, y_corpus

    def fit_transform(self, sentences):
        self.fit(sentences)
        return self.transform(sentences)

    def _transform_sentence(self, sentence, training_step=True):
        # We extract y
        if training_step:
            y = [row[self.output] for row in sentence]
        else:
            y = None

        # We pad the sentence
        sentence = self.start_rows + sentence + self.end_rows

        # We extract the features
        X = list()
        for i in range(len(sentence) - 2 * self.w_size):
            # x is a row of X
            x = list()
            # The words in lower case
            for j in range(2 * self.w_size + 1):
                if self.tolower:
                    x.append(sentence[i + j][self.input].lower())
                else:
                    x.append(sentence[i + j][self.input])
            # We represent the feature vector as a dictionary
            X.append(dict(zip(self.feature_names, x)))
        return X, y

    def print_example(self, sentences, id=1968):
        """
        :param corpus:
        :param id:
        :return:
        """
        # We print the features to check they match Table 8.1 in my book (second edition)
        # We use the training step extraction with the dynamic features
        Xs, ys = self._transform_sentence(sentences[id])
        print('X for sentence #', id, Xs)
        print('y for sentence #', id, ys)

### Running the Feature Extraction

In [ ]:
context_dictorizer = ContextDictorizer()
context_dictorizer.fit(train_dict)
X_dict, y_cat = context_dictorizer.transform(train_dict)

In [ ]:
context_dictorizer.print_example(train_dict)

### We extract all the words

In [ ]:
corpus_words = []
for x in X_dict:
    corpus_words.extend(list(x.values()))
corpus_words = sorted(list(set(corpus_words)))
print('# unique words seen in training corpus:', len(corpus_words))

### We add these words to the vocabulary
We add one word to the count for the unknown words

In [ ]:
embeddings_words = embeddings_dict.keys()
print('Words in GloVe:',  len(embeddings_dict.keys()))
vocabulary_words = set(corpus_words + list(embeddings_words))
cnt_uniq = len(vocabulary_words) + 1
print('# unique words in the vocabulary: embeddings and corpus:', 
      cnt_uniq)

### We now build an index
We keep index 0 for the unknown words

In [ ]:
word_rev_idx = dict(enumerate(vocabulary_words, start=1))
word_idx = {v: k for k, v in word_rev_idx.items()}

### We replace the words with their index

In [ ]:
for x_dict in X_dict:
    for word in x_dict:
        x_dict[word] = word_idx[x_dict[word]]
dict_vect = DictVectorizer()
X = dict_vect.fit_transform(X_dict)

print('X shape', X.shape)
print('First line of X:', X[0])

### And $\mathbf{y}$

In [ ]:
# The POS and the number of different POS
pos_list = sorted(list(set(y_cat)))
NB_CLASSES = len(pos_list) + 1

# We build a part-of-speech index. We keep 0 for unknown symbols in the test set
pos_rev_idx = dict(enumerate(pos_list, start=1))
pos_idx = {v: k for k, v in pos_rev_idx.items()}

# We encode y
y = [pos_idx[i] for i in y_cat]
print(y_cat[:10])
y[:10]

### We now create the matrix

In [ ]:
embedding_matrix = np.random.random((len(vocabulary_words) + 1, 
                                     EMBEDDING_DIM))

In [ ]:
for word in vocabulary_words:
    if word in embeddings_dict:
        # If the words are in the embeddings, we fill them with a value
        embedding_matrix[word_idx[word]] = embeddings_dict[word]

In [ ]:
model = models.Sequential()
model.add(layers.Embedding(cnt_uniq, EMBEDDING_DIM, 
                                input_length=2 * W_SIZE + 1))
if embedding_matrix is not None:
    model.layers[0].set_weights([embedding_matrix])
    model.layers[0].trainable = False
model.add(layers.Flatten())
model.add(layers.Dense(NB_CLASSES, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',
                   optimizer=OPTIMIZER,
                   metrics=['accuracy'])
model.summary()
model.fit(X, y, epochs=EPOCHS, batch_size=BATCH_SIZE)

### Testing the Model

In [ ]:
X_test_dict, y_test_cat = context_dictorizer.transform(test_dict)
for x_dict_test in X_test_dict:
    for word in x_dict_test:
        x_dict_test[word] = word_idx.get(x_dict_test[word], 0)
            
# We transform the symbols into numbers
X_test = dict_vect.transform(X_test_dict)
y_test = [pos_idx.get(i, 0) for i in y_test_cat]

test_loss, test_acc = model.evaluate(X_test, y_test)
print('Optimizer', OPTIMIZER, 'Epochs', EPOCHS, 'Batch size', 
      BATCH_SIZE, 'Mini corpus', MINI_CORPUS)
print('Loss:', test_loss)
print('Accuracy:', test_acc)

## Prediction of parts of speech

In [ ]:
def predict_sentence(sentence, dict_vect, model, word_idx, pos_rev_idx):
    column_names = ['id', 'form']
    sentence = list(enumerate(sentence.lower().split(), start=1))
    conll_cols = ''
    for tuple in sentence:
        conll_cols += str(tuple[0]) + '\t' + tuple[1] + '\n'
    # print(conll_cols)

    conll_dict = CoNLLDictorizer(column_names, col_sep='\t')
    sent_dict = conll_dict.transform(conll_cols)
    # print('Sentence:', sent_dict[0])

    context_dictorizer = ContextDictorizer()
    context_dictorizer.fit(sent_dict)
    X_dict, y = context_dictorizer.transform(sent_dict, 
                                             training_step=False)
    # print('Sentence, padded:', X_dict)
    # print('POS, y:', y)

    for x_dict in X_dict:
        for word in x_dict:
            x_dict[word] = word_idx.get(x_dict[word], 1)
    X = dict_vect.transform(X_dict)

    # print(X)
    y_prob = model.predict(X)
    y_pred = y_prob.argmax(axis=-1)
    y_pred_cat = [pos_rev_idx[i] for i in y_pred]
    return y_pred_cat

In [ ]:
sentences = ["That round table might collapse .",
                 "The man can learn well .",
                 "The man can swim .",
                 "The man can simwo ."]
for sentence in sentences:
    y_test_pred_cat = predict_sentence(sentence.lower(), 
                                       dict_vect,
                                       model, word_idx, 
                                       pos_rev_idx)
    print(sentence)
    print(y_test_pred_cat)